In [ ]:
key = ''
import requests
import json, os
import urllib.request
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
import shapely
import pickle
import random
import numpy as np
import gmaps

dataDir = "/data/gmaps_data

In [ ]:
def plotMap(polyGrid, Loc=[], restrict=[]):
    gPoly = []
    sLoc = []
    sHtext = []
    if len(restrict)==0:
        restrict = list(polyGrid.keys())
        
    for k,v in polyGrid.items():
        if k in restrict:
            gPoly.append(gmaps.Polygon(
                            list(v),
                            stroke_color='yellow',
                            fill_color='green'
                            ))
            sLoc.append((v[0][0],v[0][1]))
            sHtext.append(str(k))
    
    if len(Loc)!=0:
        sLoc = []
        sHtext = []
        for k,v in Loc.items():
            if k in restrict:
                sLoc += v
                sHtext += [str(k)+str(i) for i in v]
        
    fig = gmaps.figure(center=(39.50,-98.35), zoom_level=4, map_type='TERRAIN')
    fig.add_layer(gmaps.drawing_layer(features=gPoly))
    fig.add_layer(gmaps.symbol_layer(sLoc,info_box_content = sHtext, scale = 1))
    return fig

In [ ]:
#Load list of polygons containing us map created in grid creation notebook
mainlandGrid = pickle.load(open("mainlandGrid.pickle",'rb'))
for i in mainlandGrid.values():
    plt.plot(i[:,1],i[:,0])
print(len(mainlandGrid))

In [ ]:
# restrict grids for which data has to collected from. Otherwise data is scraped from all grids
searchGrids = mainlandGrid.keys()
# searchGrids = list(range(235,244))
print("Seacrchin Grids: {}".format("All" if searchGrids==mainlandGrid.keys() else searchGrids))

In [ ]:
base = 'https://maps.googleapis.com/maps/api/streetview'
ext = '?size=400x200&location={}&fov=100&heading={}&radius={}&pitch=10&key={}'
print("Seacrchin Grids: {}".format("All" if searchGrids==mainlandGrid.keys() else searchGrids))
for grid,coor in mainlandGrid.items():        
    poly = Polygon(np.flip(coor))
    minx, miny, maxx, maxy = poly.bounds
    count = 0
    trials = 0
    locList = []
    if grid in searchGrids:
        saveFolder = dataDir + '/{}'.format(grid)
        if os.path.exists(saveFolder)==False:
            os.mkdir(saveFolder)
        locList = os.listdir(saveFolder)
        print("################## Searching grid {} ###################".format(grid))
        print("Save folder {}".format(saveFolder))
        print("Existing locations {}".format(locList))
        while count<20 and trials<10:
            pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            location = str(pnt.y)+','+str(pnt.x)
            if (poly.contains(pnt)) and (location not in locList):
                metaUrl = base + '/metadata' + ext.format(location, 0, 10000, key)
                r = requests.get(metaUrl).json()
                trials += 1
                print("Trial: {}, count: {}".format(trials,count))
                if r['status']=='OK' and poly.contains(Point(r['location']['lng'],r['location']['lat'])):
                    location = str(r['location']['lat'])+','+str(r['location']['lng'])
                    if (location not in locList):
                        print("Valid location found: {}".format(location))
                        locList.append(location)
                        saveFile = saveFolder + "/{}".format(location)
                        if os.path.exists(saveFile)==False:
                            os.mkdir(saveFile)

                        for heading in [0,90,180]:
                            imgUrl = base + ext.format(location, heading, 10000, key)
                            urllib.request.urlretrieve(imgUrl,saveFile+'/{}_{}.jpg'.format(heading,r['date']))
                        count += 1
                        trials = 0
                    else:
                        print("Failed trial {} location exists".format(trials))
                        print("Location {}".format(location))
                else:
                    print("Failed trial {} status or contains".format(trials))
                    print("Location {}".format(location))
        print(locList)
        print("No duplicates: {}".format(len(locList)==len(set(locList))))
        print("########## Final locations for grid {}: {} ############".format(grid, len(locList)))